In [ ]:
import pandas as pd
df = pd.read_csv("data/rummagenexrummageo.csv")
df

In [ ]:
import requests
import xmltodict
import time
import pandas as pd

def fetch_pmid_from_pmc(pmc_ids, max_retries=10):
    rar = {}
    ids = ','.join(pmc_ids)
    url = f"https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/?ids={ids}&tool=my_tool&email=my_email@example.com"
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            
            # Parse the response
            data = xmltodict.parse(response.content)
            
            # Extract PubMed ID
            if 'record' in data["pmcids"]:
                pmc_data = data["pmcids"]['record']
                if isinstance(pmc_data, list):
                    for dicte in pmc_data:
                        if "@pmid" in dicte:
                            rar[dicte["@requested-id"]] = dicte["@pmid"]
                            # lala.append(dicte["@pmid"])
                        else:
                            rar[dicte["@requested-id"]] = "Empty"

                else:
                    if "@pmid" in pmc_data:
                        rar[pmc_data["@requested-id"]] = pmc_data["@pmid"]
                    else:
                        rar[pmc_data["@requested-id"]] = "Empty"

                    
                    
            return rar
        except (TypeError,KeyError) as e:
            print(pmc_ids)
            print(len(pmc_ids))
            print(f"TypeError: {data}")
            print(f"TypeError: {pmc_data}")

            raise e
        
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}. Attempt {attempt + 1} of {max_retries}")
            if attempt < max_retries - 1:
                # Exponential backoff: wait a bit before retrying
                wait_time = 2 ** attempt  # 2^0, 2^1, 2^2, ..., 2^(attempt-1)
                print(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print("Max retries reached. Unable to fetch data.{pmc_ids}")
                return None


In [ ]:
from tqdm import tqdm
import json
import os
def process_pmc_lst(tuples_list):
    batch_size = 200  
    if os.path.exists("data/pmids_dict.json"):
        with open("data/pmids_dict.json", 'r') as f:
            all_results = json.load(f)
    else:
        all_results = {}

    tuples_list.difference_update(all_results.keys())
    tuples_list = list(tuples_list)

    total_batches = (len(tuples_list) + batch_size - 1) // batch_size
    for i in tqdm(range(0, len(list(tuples_list)), batch_size), total=total_batches, desc="Processing Batches"):
        pmc_ids= tuples_list[i:i + batch_size]
        pmc_ids = set(pmc_ids) 
        pmc_ids.difference_update(all_results.keys())
        if len(pmc_ids) > 0:
            r = fetch_pmid_from_pmc(pmc_ids)
            all_results.update(r)
        with open("data/pmids_dict.json", 'w') as f:
            json.dump(all_results, f, indent=4)
        time.sleep(1)  # Delay to avoid hitting rate limits
    return all_results



In [ ]:
import os
import shutil

def empty_directory(directory_path):
    """
    Empties the specified directory if it isn't empty.

    Parameters:
    directory_path (str): The path to the directory to be emptied.
    """
    if not os.path.exists(directory_path):
        print(f"The directory {directory_path} does not exist.")
        return
    
    if not os.path.isdir(directory_path):
        print(f"{directory_path} is not a directory.")
        return
    
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Removed file: {file_path}")
            
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        
        except Exception as e:
            print(f"Failed to remove {file_path}: {e}")



In [ ]:
import GEOparse


def get_metadata_from_geo(gse_id):
    for i in range(10):
        try:
            metadata = GEOparse.get_GEO(geo=gse_id, silent=True, destdir="GEO")
            metadata = metadata.metadata
            
            if isinstance(metadata, str) and "Error: Download failed due to " in metadata:
                raise IOError()
            
            empty_directory("GEO")
            return metadata
        
        except IOError as e:
            print(f"Attempt {i+1}/10: Error downloading GEO data: {e}")
            
            if i == 9:  # On the last attempt, write to the file
                with open("data/GSEs_to_delete.txt", 'a') as file:
                    file.write(f"{gse_id}\n")
            
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            # Handle or log the unexpected error as needed

In [ ]:
import pandas as pd

def filter_by_dict(df, col1, col2, lookup_dict):
    """
    Filters out rows from the DataFrame where the value in `col1` is present 
    in the dictionary values whose key is the value in `col2`. Goal is to remove redundant studies

    Returns:
    pd.DataFrame: The filtered DataFrame.
    """
    def row_should_keep(row):
        key = row[col2]
        value = row[col1]
        # Check if the value is in the dictionary's value for the given key
        return not (key in lookup_dict and value in lookup_dict[key])
    
    tqdm.pandas()  
    df_filtered = df[df.progress_apply(row_should_keep, axis=1)]    
    return df_filtered



In [ ]:
df["geo_gse"] = df["rummageo"].str.split("-").str[0]
df["geo_gse"] = df["geo_gse"].str.split(",").str[0]
df["pmc_id"] = df["rummagene"].str.split("-").str[0]
df

In [ ]:
import json
from tqdm import tqdm
def process_geos_meta(lst):
    lst = set(lst)
    if os.path.exists("data/gse_results.json"):
        with open("data/gse_results.json", 'r') as f:
            all_results = json.load(f)
    else:
        all_results = {}
    lst.difference_update(all_results.keys())
    lst = list(lst)
    if len(lst) > 0:
        for ele in tqdm(lst, desc="Processing items"):
            all_results[ele] = get_metadata_from_geo(ele)
            with open("data/gse_results.json", 'w') as f:
                json.dump(all_results, f, indent=4)
    return


def process_geos_pmids(lst):
    lst = set(lst)

    if not os.path.exists("data/gse_results.json"):
        print("Error: File 'data/gse_results.json' does not exist.")
        return

    with open("data/gse_results.json", 'r') as f:
        all_res = json.load(f)

    try:
        pmids = {
            ele: all_res[ele].get("pubmed_id", ["Empty"])
            for ele in lst
            if isinstance(all_res.get(ele), dict)  
        }

        with open("data/pmids_dict_gse.json", 'w') as f:
            json.dump(pmids, f, indent=4)

        return 

    except Exception as e:
        print(f"An error occurred: {e}")

   

In [ ]:

process_geos_meta(list(df["geo_gse"]))


In [ ]:
try:
    with open("data/GSEs_to_delete.txt", 'r') as file:
        gse_to_delete = {line.strip() for line in file} 
        print(gse_to_delete)
    df= df[~df["geo_gse"].isin(gse_to_delete)]
    df
except:
    pass


In [ ]:
lst = set(list(df["geo_gse"]))
retrieved_list = list(lst)
process_geos_pmids(retrieved_list)

In [ ]:
pmc_pmids = process_pmc_lst(set(df["pmc_id"]))

In [ ]:
with open("data/pmids_dict.json", 'r') as f:
        all_results = json.load(f)
df['pmid'] = df['pmc_id'].map(all_results)
df

In [ ]:
with open("data/pmids_dict_gse.json", 'r') as f:
        all_results = json.load(f)
all_results


In [ ]:
df = filter_by_dict(df, "pmid", "geo_gse", all_results)
df

In [ ]:
with open("data/pmids_dict_gse.json") as f:
    gses = json.load(f)
gses = {ele:','.join(gses[ele]) for ele in gses}
df["geo_pmid"] = df["geo_gse"].map(gses)
df


In [ ]:
df  = df.drop_duplicates(subset=["rummagene","rummageo"])

In [ ]:
df.to_csv('data/rummagenexrummageo.csv', index=False)
